<a href="https://colab.research.google.com/github/Rishit2105/ecommerce-revenue-risk-sql/blob/master/python/profit_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>